In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250
dropout = 0.2

cpu


In [10]:
with open('wizard.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [11]:
char_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_char = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [char_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_char[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])


tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [12]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x, y = get_batch('train')
print(x)
print(y)

tensor([[ 1, 28, 68, 71, 68, 73, 61, 78],
        [72,  1, 56, 61, 58, 58, 64,  1],
        [54, 73,  1, 68, 67, 56, 58,  1],
        [ 1, 69, 71, 68, 74, 57,  1, 73]])
tensor([[28, 68, 71, 68, 73, 61, 78, 11],
        [ 1, 56, 61, 58, 58, 64,  1, 54],
        [73,  1, 68, 67, 56, 58,  1, 72],
        [69, 71, 68, 74, 57,  1, 73, 68]])


In [13]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [14]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [15]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        # B = batch size, T = block size, C = vocabulary size
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
    
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index) # logits = (B, T, C), loss is None
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            index = torch.cat((index, index_next), dim=1) # (B, max_new_tokens)
            
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


hhJz4k.lf
je&YtOC&Bhi9U﻿)(GUX) ASeg)4?e!YeOHV O68DJyZXMEdjoDapH
XM"1z4-RsctZ9S"-RT km"5INXtOveluD6Iuxu)(LvsS:FHWaSx  Kut_iVsxsxfm)!80Rsx9T Vi'wo"T5*VT5q5lCq7S5RACy*iQXl2(﻿h3QlD)K6_0GhJqz)27d1h3xzxpjxzln,2H*cY)'zx[eD8v1TCNvg7﻿vsV*) 9pU!0]vH*N(PE8?F1Rr8cVb zP6DSa3_Tts2?g;g;iF52&dV]vq9t8y.oD(3?tlQXTWMNrOMj"j,:9gh-bl:)eFpuX:﻿9f I4R1zbpbrv﻿0OFc_M:vclBusrM,J'b6ETCOV﻿E]_Rsr8
N2c_N4'_s;,T 3]5x﻿﻿yP"e*YjSU"PI unqJB
F?w[.9Fpp
JYnd53*Sa7
wN8E;Gb0 b-"j5kh&7Sa?7z)AqV﻿tl
T4kt3f-F)_'JF)rrV*)M:cCV0:TXH
zeWx8*kvL


In [24]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# model.parameters() -> all trainable weights in the model (here just the embedding table)
# Each parameter tensor has .data and .grad

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss {losses['train']:.3f}, val loss {losses['val']:.3f}")
    
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True) # parameter.grad is None
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss 4.809, val loss 4.806
step: 250, train loss 4.743, val loss 4.741
step: 500, train loss 4.696, val loss 4.663
step: 750, train loss 4.625, val loss 4.612
step: 1000, train loss 4.549, val loss 4.563
step: 1250, train loss 4.480, val loss 4.500
step: 1500, train loss 4.436, val loss 4.448
step: 1750, train loss 4.413, val loss 4.384
step: 2000, train loss 4.311, val loss 4.318
step: 2250, train loss 4.262, val loss 4.285
step: 2500, train loss 4.224, val loss 4.244
step: 2750, train loss 4.180, val loss 4.187
step: 3000, train loss 4.117, val loss 4.135
step: 3250, train loss 4.069, val loss 4.074
step: 3500, train loss 4.026, val loss 4.013
step: 3750, train loss 3.983, val loss 3.990
step: 4000, train loss 3.911, val loss 3.927
step: 4250, train loss 3.864, val loss 3.895
step: 4500, train loss 3.832, val loss 3.855
step: 4750, train loss 3.777, val loss 3.778
step: 5000, train loss 3.711, val loss 3.765
step: 5250, train loss 3.716, val loss 3.720
step: 5500, trai

In [1]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

NameError: name 'torch' is not defined